In [ ]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=054e65044265f07dc0b79d4d24481f3232e30510fa8aa2e2e0395f8a520487c3
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
!cd /usr/local/lib/python3.10/dist-packages/pyspark/jars && wget https://redshift-downloads.s3.amazonaws.com/drivers/jdbc/2.1.0.24/redshift-jdbc42-2.1.0.24.jar

--2024-06-18 08:02:54--  https://redshift-downloads.s3.amazonaws.com/drivers/jdbc/2.1.0.24/redshift-jdbc42-2.1.0.24.jar
Resolving redshift-downloads.s3.amazonaws.com (redshift-downloads.s3.amazonaws.com)... 52.217.123.57, 3.5.28.195, 52.217.139.121, ...
Connecting to redshift-downloads.s3.amazonaws.com (redshift-downloads.s3.amazonaws.com)|52.217.123.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1072235 (1.0M) [binary/octet-stream]
Saving to: ‘redshift-jdbc42-2.1.0.24.jar’

redshift-jdbc42-2.1 100%[===================>]   1.02M  --.-KB/s    in 0.1s    

2024-06-18 08:02:54 (8.53 MB/s) - ‘redshift-jdbc42-2.1.0.24.jar’ saved [1072235/1072235]



In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark DataFrame #5") \
    .getOrCreate()

## Redshift와 연결해서 테이블들을 데이터프레임으로 로딩하기

In [ ]:
df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:****/dev?user=guest&password=****") \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

In [ ]:
df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:****/dev?user=guest&password=****") \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

In [ ]:
df_user_session_channel.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)



In [ ]:
df_user_session_channel.rdd.getNumPartitions()

1

In [ ]:
df_session_timestamp.printSchema()

root
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [ ]:
df_session_timestamp.rdd.getNumPartitions()

1

## DataFrame으로 처리하기

In [ ]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")

In [ ]:
session_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [ ]:
session_df.show(5)

+------+--------------------+--------+--------------------+--------------------+
|userid|           sessionid| channel|           sessionid|                  ts|
+------+--------------------+--------+--------------------+--------------------+
|  1501|0135456d6a3c1051f...|  Google|0135456d6a3c1051f...|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...|Facebook|01a416a7e28d0d229...|2019-05-26 14:23:...|
|  2776|029bf49b584c641f0...|Facebook|029bf49b584c641f0...|2019-11-11 20:37:...|
|   243|0226aa5193c66d990...|  Google|0226aa5193c66d990...|2019-07-01 23:04:...|
|   939|02b8d6c2775b756de...|  Google|02b8d6c2775b756de...|2019-09-01 15:29:...|
+------+--------------------+--------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner").select(
    "userid", "sessionid", "channel", "ts"
)

AnalysisException: Reference 'sessionid' is ambiguous, could be: sessionid, sessionid.

In [ ]:
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner").select(
    "userid", df_user_session_channel.sessionid, "channel", "ts"
)

In [ ]:
channel_count_df = session_df.groupby("channel").count().orderBy("count", ascending=False)

In [ ]:
channel_count_df.show()

+---------+-----+
|  channel|count|
+---------+-----+
|  Youtube|17091|
|   Google|16982|
|    Naver|16921|
|  Organic|16904|
|Instagram|16831|
| Facebook|16791|
+---------+-----+



In [ ]:
from pyspark.sql.functions import date_format, asc, countDistinct

session_df.withColumn('month', date_format('ts', 'yyyy-MM')).groupby('month').\
    agg(countDistinct("userid").alias("mau")).sort(asc('month')).show()

+-------+---+
|  month|mau|
+-------+---+
|2019-05|281|
|2019-06|459|
|2019-07|623|
|2019-08|662|
|2019-09|639|
|2019-10|763|
|2019-11|721|
+-------+---+



## Spark SQL로  처리하기

In [ ]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")

In [ ]:
df_session_timestamp.createOrReplaceTempView("session_timestamp")

In [ ]:
channel_count_df = spark.sql("""
    SELECT channel, count(distinct userId) uniqueUsers
    FROM session_timestamp st
    JOIN user_session_channel usc ON st.sessionID = usc.sessionID
    GROUP BY 1
    ORDER BY 1
""")

In [ ]:
channel_count_df

DataFrame[channel: string, uniqueUsers: bigint]

In [ ]:
channel_count_df.show()

+---------+-----------+
|  channel|uniqueUsers|
+---------+-----------+
| Facebook|        889|
|   Google|        893|
|Instagram|        895|
|    Naver|        882|
|  Organic|        895|
|  Youtube|        889|
+---------+-----------+



In [ ]:
mau_df = spark.sql("""
SELECT
  LEFT(A.ts, 7) AS month,
  COUNT(DISTINCT B.userid) AS mau
FROM session_timestamp A
JOIN user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1
ORDER BY 1 DESC""")

In [ ]:
mau_df.collect()

[Row(month='2019-11', mau=721),
 Row(month='2019-10', mau=763),
 Row(month='2019-09', mau=639),
 Row(month='2019-08', mau=662),
 Row(month='2019-07', mau=623),
 Row(month='2019-06', mau=459),
 Row(month='2019-05', mau=281)]